In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely import Point
from shapely.ops import nearest_points
from rtree import index
from tqdm import tqdm
from shapely.geometry import Point, Polygon, LineString,MultiLineString

In [2]:
with open(r'nodes_edges_ucla_access.pickle', 'rb') as handle:
    B_matrix_sliced,B_matrix_str_sliced,nodes_coordinates_array = pickle.load(handle)

# with open(r'nodes_edges_ucla_big_graph.pickle', 'rb') as handle:
#     B_matrix_sliced,B_matrix_str_sliced,nodes_coordinates_array = pickle.load(handle)


# udf = gpd.read_file('uclaaccess_roi_widths_short.geojson')

# blocks=gpd.read_file(r"blocks_epsg4326.geojson")
blocks=gpd.read_file(r"blocks_epsg3857.geojson")


/var/folders/d3/99lvl9fd1673ngz9966pvq100000gn/T/ipykernel_6562/3997957124.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  B_matrix_sliced,B_matrix_str_sliced,nodes_coordinates_array = pickle.load(handle)


In [5]:
# Import and subset block data
blocknumbers = np.array(blocks["GEOID20"],dtype=str)
blockboundaries = np.array(blocks["geometry"])

In [6]:
Hillside_NodeCoordinate = nodes_coordinates_array[:,0:2]

In [8]:
# Create a dictionary mapping block boundaries to their indices
block_to_index = {boundary: i for i, boundary in enumerate(blockboundaries)}

# Use a set to keep track of identified nodes
identified_nodes = set()

# Use a DataFrame for Node_Block
Node_Block = pd.DataFrame(columns=['Node', 'BlockNumber', 'BlockBoundary'])

In [9]:
# Create an R-tree index
idx = index.Index()
for i, boundary in enumerate(blockboundaries):
    idx.insert(i, boundary.bounds)

In [14]:
# Identify nodes that are in a block using R-tree
for i in tqdm(range(len(Hillside_NodeCoordinate)), desc="Filtering nodes"):
    coord = Hillside_NodeCoordinate[i]
    possible_blocks_indices = list(idx.intersection((coord[0], coord[1], coord[0], coord[1])))
    for block_index in possible_blocks_indices:
        block = blockboundaries[block_index]
        if block.contains(Point(coord[0], coord[1])):
            new_row = pd.DataFrame({'Node': [i], 'BlockNumber': [blocknumbers[block_index]], 'BlockBoundary': [block]})
            Node_Block = pd.concat([Node_Block, new_row], ignore_index=True)
            identified_nodes.add(i)

Filtering nodes:   1%|          | 1093/199504 [00:00<00:35, 5536.38it/s]

[12559, 71219, 85478]
Node 0 is in block 060378002022001
[12559, 71220, 12101, 85478]
Node 1 is in block 060378002022007
[75829, 75827]
Node 2 is in block 060378001041062
[12101, 50322, 73107, 75112, 75422, 83957]
Node 3 is in block 060378002061016
[12559, 12101, 9610]
Node 4 is in block 060378002022000
[12559, 71220, 85478]
Node 5 is in block 060378002022001
[76495, 32476]
Node 6 is in block 060378005041012
[12559, 12101, 9610]
Node 7 is in block 060378002022000
[12559, 71220, 85478]
Node 8 is in block 060378002022001
[12559, 71220, 85478]
Node 9 is in block 060378002022001
[4961, 54928, 4965]
Node 10 is in block 060378002021014
[76495, 32476]
Node 11 is in block 060378005041012
[75829]
Node 12 is in block 060378005041003
[12559, 12101, 9610, 85478]
Node 13 is in block 060378002022000
[12559, 71220, 85478]
Node 14 is in block 060378002022007
[12559, 71220, 12101, 9610, 85478]
Node 15 is in block 060378002022007
[4961, 54928, 4965]
Node 16 is in block 060378002021014
[12559, 12101, 961

Filtering nodes:   1%|          | 1647/199504 [00:00<00:39, 5067.11it/s]

Node 1093 is in block 060378002041016
[27564, 27452, 7235, 50328, 84094]
Node 1094 is in block 060378002061019
[27564, 27452, 22850, 7235, 50328, 84094]
Node 1095 is in block 060378002061019
[27564, 27452, 22850, 7235, 50328, 84094]
Node 1096 is in block 060378002061019
[51703, 64656]
Node 1097 is in block 060378001041034
[7235, 18105, 73000, 84094]
Node 1098 is in block 060378002041008
[85482]
Node 1099 is in block 060378001041037
[75695, 8509]
Node 1100 is in block 060371344242001
[81508, 64698]
Node 1101 is in block 060378002023000
[66614, 60500]
Node 1102 is in block 060371132311015
[66614, 60500]
Node 1103 is in block 060371132311016
[12101, 7235, 18105, 73000]
Node 1104 is in block 060378002041015
[51153, 63399, 60530]
Node 1105 is in block 060371344213002
[75695, 8509]
Node 1106 is in block 060371344242003
[12101, 10240, 82795, 81517]
Node 1107 is in block 060378002041016
[46286, 66614, 60500, 29311]
Node 1108 is in block 060371132311016
[4961, 1194]
Node 1109 is in block 060378

Filtering nodes:   1%|▏         | 2798/199504 [00:00<00:35, 5475.12it/s]

...

Filtering nodes:   5%|▌         | 10053/199504 [00:01<00:38, 4866.42it/s]

[3714, 66177, 57427]
Node 9250 is in block 060371133031012
[9858]
Node 9251 is in block 060371133031013
[63381]
Node 9252 is in block 060371347203001
[63381]
Node 9253 is in block 060371347203001
[70534, 4610, 66803]
Node 9254 is in block 060378005063003
[65969, 8394]
Node 9255 is in block 060371349052003
[8394, 69559]
Node 9256 is in block 060371349052000
[7809, 53837, 84670]
Node 9257 is in block 060371082022001
[8394, 69559]
Node 9258 is in block 060371349052003
[45331]
Node 9259 is in block 060371082021008
[14926, 77419]
Node 9260 is in block 060371082021005
[11626]
Node 9261 is in block 060371348012004
[31852, 50185, 11525]
Node 9262 is in block 060371131022004
[7809, 24620]
Node 9263 is in block 060371082022014
[3714, 3695]
Node 9264 is in block 060371133031015
[11626]
Node 9265 is in block 060371348012004
[13045]
Node 9266 is in block 060371348012001
[81348]
Node 9267 is in block 060371341042000
[5749, 81348]
Node 9268 is in block 060371341042000
[50204, 61005]
Node 9269 is in b

Filtering nodes:   5%|▌         | 10347/199504 [00:02<00:37, 5065.95it/s]


Node 10220 is in block 060372626011008
[12542, 41431, 64682]
Node 10221 is in block 060371375041008
[9835, 64505]
Node 10222 is in block 060371082041000
[14402, 91068]
Node 10223 is in block 060371133031000
[14402]
Node 10224 is in block 060371133031000
[65865]
Node 10225 is in block 060371394021008
[7809, 5544, 5535, 82723]
Node 10226 is in block 060371082031011
[7809, 9835, 9966, 5544, 3836, 82723]
Node 10227 is in block 060371082031014
[5697, 32197]
Node 10228 is in block 060371134231002
[9835, 14344]
Node 10229 is in block 060371082041000
[2843, 11589, 53179]
Node 10230 is in block 060371393011000
[81091, 14587, 45468, 64447, 47115]
Node 10231 is in block 060372626011008
[71077, 78666]
Node 10232 is in block 060371375041000
[70597, 22996, 34882, 82115]
Node 10233 is in block 060372626043013
[62777, 86776]
Node 10234 is in block 060371134242003
[41431, 64682]
Node 10235 is in block 060371375041008
[5704, 62777, 86776]
Node 10236 is in block 060371134242003
[81091, 14587, 45468, 6444

KeyboardInterrupt: 

In [32]:
# Identify nodes that are not in any block
unidentified_nodes = set(range(len(Hillside_NodeCoordinate))) - identified_nodes

In [34]:
len(unidentified_nodes)

199504